# Imports

In [1]:
!pip install braindecode
!pip install moabb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 70.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.0/365.0 kB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyriemann: filename=pyriemann-0.3-py2.py3-none-any.whl size=78031 sha256=29d68c0350eccd5890ea9c345da9c55a9c2fc89e3ac94c2e5da6e9b5a6784dbc
  Stored in directory: /root/.cache/pip/wheels/eb/52/63/ad042f5ca1209b213a326a843e75d730b30bc7a89a79edb187
Successfully built pyriemann
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.2
    Uninstalling pandas-2.0.2:
      Successfully uninstalled pandas-2.0.2
ERROR: pip's depe

In [2]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms, utils
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F


import matplotlib.pyplot as plt
# torch.manual_seed(0) # Set for our testing purposes, please do not change!

from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import (
    exponential_moving_standardize, preprocess, Preprocessor)
from braindecode.preprocessing import \
    create_windows_from_events, create_fixed_length_windows
import pandas as pd
import numpy as np


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Tensorflow not install, you could not use those pipelines


In [3]:
import torch
from torch.utils.data import Dataset
import numpy as np


class EEG(Dataset):

    def __init__(self, 
                 subject_id = 3, 
                 dataset_name="BNCI2014001", 
                 channels = ['C3', 'Cz', 'C4'],
                 transform = None):
        
        self.raw_dataset     = MOABBDataset(dataset_name = dataset_name, subject_ids=subject_id)
        self.prepro_dataset  = preprocessor(self.raw_dataset)
        self.windows_dataset = get_windows(self.prepro_dataset, picks = channels)
        self.data            = get_tensors_from_windows(self.windows_dataset)
        self.transform       = transform
        self.classes         = self.windows_dataset.datasets[0].windows.event_id
        
    def __len__(self):
        return self.data[0].shape[0]


    def __getitem__(self,idx):

        # sample = {'signal': torch.from_numpy(self.data[0])[idx], 'label': torch.from_numpy(self.data[1])[idx]}

        sample = (torch.from_numpy(np.expand_dims(self.data[0], axis = 1))[idx], torch.from_numpy(self.data[1])[idx])

        if self.transform:
            sample = self.transform(sample)

        return sample


def get_noise(n_samples, input_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, input_dim)
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        input_dim: the dimension of the input vector, a scalar
        device: the device type
    '''
    return torch.randn(n_samples, input_dim, device=device)


def get_one_hot_labels(labels, n_classes):
    '''
    Function for creating one-hot vectors for the labels, returns a tensor of shape (?, num_classes).
    Parameters:
        labels: tensor of labels from the dataloader, size (?)
        n_classes: the total number of classes in the dataset, an integer scalar
    '''
    return F.one_hot(labels,n_classes)

def combine_vectors(x, y):
    '''
    Function for combining two vectors with shapes (n_samples, ?) and (n_samples, ?).
    Parameters:
      x: (n_samples, ?) the first vector. 
        In this assignment, this will be the noise vector of shape (n_samples, z_dim), 
        but you shouldn't need to know the second dimension's size.
      y: (n_samples, ?) the second vector.
        Once again, in this assignment this will be the one-hot class vector 
        with the shape (n_samples, n_classes), but you shouldn't assume this in your code.
    '''
    combined = torch.cat((x.float(),y.float()), 1)
    return combined

def get_input_dimensions(z_dim, eeg_shape, n_classes):
    '''
    Function for getting the size of the conditional input dimensions 
    from z_dim, the image shape, and number of classes.
    Parameters:
        z_dim: the dimension of the noise vector, a scalar
        eeg_shape: the shape of each EEG data as (C, W, H), which is (1, 3, 400), that is, we choose 3 channels (3 electrodes)
        n_classes: the total number of classes in the dataset, an integer scalar
                (4 for EEG, that is 4 movements - tongue, left hand, right hand and feet)
    Returns: 
        generator_input_dim: the input dimensionality of the conditional generator, 
                          which takes the noise and class vectors
        discriminator_im_chan: the number of input channels to the discriminator
                            (e.g. C x 3 x 400 for EEG)
    '''
    generator_input_dim = z_dim + n_classes
    discriminator_im_chan = eeg_shape[0] + n_classes
    return generator_input_dim, discriminator_im_chan

def weights_init(m):
    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, torch.nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)


# preprocess function
def preprocessor(
    dataset,
    low_cut_hz = 4.,   # low cut frequency for filtering
    high_cut_hz = 38., # high cut frequency for filtering
    newfreq = 100, # Paramater for resampling
    factor = 1e6, # Parameter for scaling
    ):

    preprocessors = [
        Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
        Preprocessor(lambda data: np.multiply(data, factor)),  # Convert from V to uV
        Preprocessor("resample", sfreq=newfreq), # Resampling
        Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
        Preprocessor("set_eeg_reference", ref_channels="average", ch_type="eeg") # Common Average Reference
    ]
    
    # Transform the data
    # return preprocess(dataset, preprocessors, n_jobs = -1)
    return preprocess(dataset, preprocessors)



# Get Windows from Dataset

def get_windows(
        dataset, 
        trial_start_offset_samples=0,
        trial_stop_offset_samples=100,
        window_size_samples=400,
        window_stride_samples=100,
        preload=True,
        # mapping = {'left_hand': 0, 'right_hand': 1},
        picks = ['C3', 'Cz', 'C4']
        ):
    
    windows_dataset = create_windows_from_events(
        dataset,
        trial_start_offset_samples = trial_start_offset_samples,
        trial_stop_offset_samples  = trial_stop_offset_samples,
        window_size_samples        = window_size_samples,
        window_stride_samples      = window_stride_samples,
        preload                    = True,
        # mapping = {'left_hand': 0, 'right_hand': 1},
        picks                      = picks
        )
    
    return windows_dataset


def get_tensors_from_windows(windows_dataset):
    windows_list = []
    labels_list = []
    n_runs = len(windows_dataset.datasets)
    for i in range(n_runs):
        windows_list.append(windows_dataset.datasets[i].windows.get_data())
        labels_list.append(windows_dataset.datasets[i].y)
        
    stacked_tensor = np.concatenate(windows_list, axis=0)
    stacked_labels = np.concatenate(labels_list, axis=0)
        
    return stacked_tensor, stacked_labels





##### GENERATOR #####
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        input_dim: the dimension of the input vector, a scalar
        im_chan: the number of channels of the output eeg, a scalar
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, input_dim=68, im_chan=1, hidden_dim=64):
        super(Generator, self).__init__()
        self.input_dim = input_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(input_dim, hidden_dim * 4,      kernel_size = (1,60), stride = (1,1)),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size = (1,60), stride = (1,1)),
            self.make_gen_block(hidden_dim * 2, hidden_dim,     kernel_size = (1,60), stride = (1,1)),
            self.make_gen_block(hidden_dim, im_chan,            kernel_size = (3,50), stride = (1,2), padding = (0,2), final_layer=True),
        )

    def make_gen_block(self, input_channels, output_channels, kernel_size, stride, padding = 0, final_layer=False):
        '''
        Function to return a sequence of operations corresponding to a generator block of DCGAN;
        a transposed convolution, a batchnorm (except in the final layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation has
            output_channels: how many channels the output feature representation should have
            kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false otherwise 
                      (affects activation and batchnorm)
        '''
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
                nn.Tanh(),
            )

    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, input_dim)
        '''
        x = noise.view(len(noise), self.input_dim, 1, 1)
        return self.gen(x)


##### Discriminator #####
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
      im_chan: the number of channels of the output eeg, a scalar
      hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_chan=5, hidden_dim=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            self.make_disc_block(im_chan, hidden_dim,        kernel_size = (1,50), stride = (2,4)),
            self.make_disc_block(hidden_dim, hidden_dim * 2, kernel_size = (1,50), stride = (2,4)),
            self.make_disc_block(hidden_dim * 2, 1,          kernel_size = (1,10), stride = (2,1), final_layer=True),
        )

    def make_disc_block(self, input_channels, output_channels, kernel_size, stride, final_layer=False):
        '''
        Function to return a sequence of operations corresponding to a discriminator block of the DCGAN; 
        a convolution, a batchnorm (except in the final layer), and an activation (except in the final layer).
        Parameters:
            input_channels: how many channels the input feature representation has
            output_channels: how many channels the output feature representation should have
            kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false otherwise 
                      (affects activation and batchnorm)
        '''
        if not final_layer:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.LeakyReLU(0.2, inplace=True),
            )
        else:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
            )

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: Given an image tensor, 
        returns a 1-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_chan)
        '''
        disc_pred = self.disc(image)
        return disc_pred.view(len(disc_pred), -1)



In [4]:
# from eeg import EEG
# from MyGAN import *
# from gan_input_functions import *
# from helper_functions import *


import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms, utils
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch.optim as optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
# torch.manual_seed(0) # Set for our testing purposes, please do not change!

import pandas as pd
import numpy as np

# Getting data

In [5]:
eeg_data = EEG(subject_id=[1])

/opt/conda/lib/python3.10/site-packages/moabb/datasets/download.py:54: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_BNCI_PATH"
  set_config(key, get_config("MNE_DATA"))


MNE_DATA is not already configured. It will be set to default location in the home directory - /root/mne_data
All datasets will be downloaded to this location, if anything is already downloaded, please move manually to this location


100%|█████████████████████████████████████| 42.8M/42.8M [00:00<00:00, 49.7GB/s]
SHA256 hash of downloaded file: 054f02e70cf9c4ada1517e9b9864f45407939c1062c6793516585c6f511d0325
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|█████████████████████████████████████| 43.8M/43.8M [00:00<00:00, 49.8GB/s]
SHA256 hash of downloaded file: 53d415f39c3d7b0c88b894d7b08d99bcdfe855ede63831d3691af1a45607fb62
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.

/opt/conda/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB sto

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand',

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


# Configuration

In [6]:
# n_epochs = 500
z_dim = 64
# display_step = 500
# lr = 0.00001
n_classes = 4
# batch_size = 32
device = 'cuda'

In [7]:
# PATH = "../Weights/Conv_Cond_Gan_v2/Gen_Weights_v2.pt"

In [35]:
gen = Generator()

In [41]:
%mkdir weights
%cd weights

In [76]:
# !gdown https://drive.google.com/uc?1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55
!gdown "https://drive.google.com/file/d/1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55/view?usp=share_link" -O weights.pt

/opt/conda/lib/python3.10/site-packages/gdown/parse_url.py:44: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55/view?usp=share_link
To: /root/weights/weights.pt
83.7kB [00:00, 329MB/s]


In [77]:
!ls

 1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55	    'view?usp=share_link'
 index.html				    'view?usp=share_link.1'
'uc?1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55'	     weights.pt
'uc?id[1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55]'


In [73]:
!ls

/root/weights


In [78]:
gen.load_state_dict(torch.load("./weights.pt"))

UnpicklingError: invalid load key, '<'.

## Generate Samples

In [10]:



def generate_samples_with_labels(label, n_samples, generator, z_dim = 64, channel = None, extra_dim = True):
    '''
    Function for generating samples, once the generator has been trained
        label: label of the movement to be sampled. See dictionary below
        {'feet': 0, 'left_hand': 1, 'right_hand': 2, 'tongue': 3}
        n_samples: number of samples to be generated
        channel: electrode {'C3': 0, 'Cz': 1, 'C4': 2} -> Default: All channels
        generator: the trained generator
    '''

    n_classes = 4

    if channel == None:
        noise_4_gen = get_noise(n_samples, z_dim)
        label = get_one_hot_labels(torch.Tensor([label]).long(), n_classes).repeat(n_samples,1)

        noise_and_labels = combine_vectors(noise_4_gen, label)
        fake = generator(noise_and_labels)

        if extra_dim == False:
            fake = fake.reshape((fake.shape[0], fake.shape[2], fake.shape[3]))
        return fake
    else:
        noise_4_gen = get_noise(n_samples, z_dim)
        label = get_one_hot_labels(torch.Tensor([label]).long(), n_classes).repeat(n_samples,1)

        noise_and_labels = combine_vectors(noise_4_gen, label)
        fake = generator(noise_and_labels)
        filtered_channel_fake = torch.select(fake, dim = 2, index = channel)

        if extra_dim == False:
            filtered_channel_fake = filtered_channel_fake.reshape((filtered_channel_fake.shape[0], filtered_channel_fake.shape[2]))

        return filtered_channel_fake



## Filter Label and channels of real data

In [11]:
def filter_label_and_channel(eeg_data, label, channel):
    '''
    Function to filter label and channel of original eeg data.
        eeg_data: raw eeg data
        label: class of movement
        channel: electrode --   {'C3': 0, 'Cz': 1, 'C4': 2}
    '''


    mask = torch.where(eeg_data[:][1] == label, 1, 0)
    filtered_eeg = eeg_data[:][0][torch.nonzero(mask).flatten()]
    filtered_eeg = filtered_eeg.reshape((filtered_eeg.shape[0], filtered_eeg.shape[2], filtered_eeg.shape[3] ))
    filtered_channel_eeg = torch.select(filtered_eeg, dim = 1, index = channel)

    return filtered_channel_eeg

## Classification Helper Functions

In [12]:
def generate_samples_for_classification(n_samples, generator, z_dim = 64):
    '''
    Function for generating equal label samples for the classifier.
        n_samples: number of samples to be generated
        generator: the trained generator
    '''
    n_classes = 4

    n_samples_partial = int(n_samples/n_classes)
    noise_4_gen = get_noise(n_samples, z_dim)

    label = [0,1,2,3]

    label = [get_one_hot_labels(torch.Tensor([i]).long(), n_classes).repeat(n_samples_partial,1) for i in label]

    label_concat = torch.zeros_like(torch.Tensor(0,4))
    for i in range(len(label)):
        label_concat = torch.cat((label_concat,label[i]), 0)

    noise_and_labels = combine_vectors(noise_4_gen, label_concat)

    fake = generator(noise_and_labels)

    original_labels = torch.argmax(label_concat,dim = 1)
    return (fake,original_labels)


In [13]:
def add_real_fake(real_eeg, fake_eeg):

    real_data = real_eeg[:]
    fake_data = fake_eeg

    complete_eeg_data = torch.cat((real_data[0], fake_data[0]), dim = 0)
    complete_label_data = torch.cat((real_data[1], fake_data[1]), dim = 0)

    return (complete_eeg_data, complete_label_data)


## Classifier

In [14]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.T = 400

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 400), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.tconv2d1 = nn.ConvTranspose2d(16, 16, kernel_size = (118,1), stride = (1,1))
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 2))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 1)

        # # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d(2, 1)

        # # # FC Layer
        # # # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # # # I have 120 timepoints.
        self.fc1 = nn.Linear(4*14*33, 4)


    def forward(self, x):

        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = self.conv2(x)
        x = F.elu(x)
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)

        # FC Layer
        x = x.reshape(-1, 4*14*33)
        x = F.softmax(self.fc1(x), dim = 1)
        
        return x

class LinClassifier(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(LinClassifier,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

        
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

In [15]:
## data
X = eeg_data[:][0].numpy()
# X = torch.select(eeg_data[:][0], 2, 0).numpy()
Y = F.one_hot(eeg_data[:][1], n_classes).numpy()

In [ ]:
# Get weights from internet

In [32]:
!wget https://drive.google.com/file/d/1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55/view?usp=share_link

--2023-10-17 23:06:47--  https://drive.google.com/file/d/1YX5kKm1ryJxh6ZAOEttlnr__RAfeni55/view?usp=share_link
Resolving drive.google.com (drive.google.com)... 74.125.201.139, 74.125.201.102, 74.125.201.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.201.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=share_link’

view?usp=share_link     [ <=>                ]  81.75K  --.-KB/s    in 0.002s  

2023-10-17 23:06:47 (39.4 MB/s) - ‘view?usp=share_link’ saved [83714]



In [33]:
!ls

'view?usp=share_link'


In [16]:
# Complete REAL+FAKE
# fake_data = generate_samples_for_classification(X.shape[0], gen)
# complete_dataset = add_real_fake(eeg_data,fake_data)
# X = complete_dataset[:][0].detach().numpy()
# # X = torch.select(eeg_data[:][0], 2, 0).numpy()
# Y = F.one_hot(complete_dataset[:][1], n_classes).numpy()

In [17]:
### Train test split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=42)
x_train = x_train.astype("float32")
x_val = x_val.astype("float32")

y_train = y_train.astype("float32")
y_val = y_val.astype("float32")


## transforming tensors
x_val = torch.from_numpy(x_val).cuda()
y_val = torch.from_numpy(y_val).cuda()

In [18]:
# DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x_train).cuda(0)
        self.y=torch.from_numpy(y_train).cuda(0)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [19]:
n_epochs = 10
lr = 0.00001
batch_size = 32

In [20]:
data_set=Data()
trainloader = DataLoader(dataset=data_set,batch_size=batch_size)
data_set.x.shape, data_set.y.shape

(torch.Size([806, 1, 3, 400]), torch.Size([806, 4]))

In [21]:
classif = Classifier().cuda(0)
# criterion = F.cross_entropy()
criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classif.parameters(), lr = lr)

## Train

In [22]:
# loss_list      = []
# acc_train_list = []
# acc_val_list   = []

# step = 0
# progress_show = 20

# #n_epochs
# for epoch in range(n_epochs):
    
#     for x, y in trainloader:
        
#         # Send variables to GPU
#         x, y = x.cuda(0), y.cuda(0)
        
#         #clear gradient 
#         optimizer.zero_grad()
#         #make a prediction 
#         z=classif(x)
#         # calculate loss, da Cross Entropy
#         loss=criterion(z,y)
#         # # calculate gradients of parameters 
#         loss.backward()
#         # # update parameters 
#         optimizer.step()
        
#         loss_list.append(round(loss.data.item(), 3))

#     # Train Evaluation
#     acc_train = ((torch.argmax(z, 1) == torch.argmax(y, 1)).float().mean()).item()
#     acc_train = round(acc_train, 3)
#     acc_train_list.append(acc_train)

#     # Validation Evaluation
#     z_val = classif(x_val)
#     acc_val = ((torch.argmax(z_val, 1) == torch.argmax(y_val, 1)).float().mean()).item()
#     acc_val = round(acc_val, 3)
#     acc_val_list.append(acc_val)

#     # if acc_val > best_acc_val:
#         # torch.save(classif, 'best-model.pt') 
#         # print(acc_val)

        
#     best_acc_val = max([i for i in acc_val_list])
    
#     # if step % progress_show == 0:
#     #     print('epoch {}, loss {}, acc_train {}, acc_val {}'.format(epoch, loss.item(), acc_train,  acc_val))


#     step += 1


## Test parameters

In [30]:
def train(lr, batch_size):

    n_epochs = 500

    data_set=Data()
    trainloader = DataLoader(dataset=data_set,batch_size=batch_size)

    classif = Classifier().cuda(0)
    # criterion = F.cross_entropy()
    criterion = nn.BCELoss()
    # criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classif.parameters(), lr = lr)
    
    loss_list      = []
    acc_train_list = []
    acc_val_list   = []
    
#     step = 0
#     progress_show = 20
    
    #n_epochs
    for epoch in range(n_epochs):
        for x, y in (trainloader):
            
            # Send variables to GPU
            x, y = x.cuda(0), y.cuda(0)
            
            #clear gradient 
            optimizer.zero_grad()
            #make a prediction 
            z=classif(x)
            # calculate loss, da Cross Entropy
            loss=criterion(z,y)
            # # calculate gradients of parameters 
            loss.backward()
            # # update parameters 
            optimizer.step()
            
            loss_list.append(round(loss.data.item(), 3))
    
        # Train Evaluation
        acc_train = ((torch.argmax(z, 1) == torch.argmax(y, 1)).float().mean()).item()
        acc_train = round(acc_train, 3)
        acc_train_list.append(acc_train)
    
        # Validation Evaluation
        z_val = classif(x_val)
        acc_val = ((torch.argmax(z_val, 1) == torch.argmax(y_val, 1)).float().mean()).item()
        acc_val = round(acc_val, 3)
        acc_val_list.append(acc_val)
        best_acc_val = max([i for i in acc_val_list])

#         if step % progress_show == 0:
#             print(f"Epoch {epoch}")
        
        # if acc_val > best_acc_val:
            # torch.save(classif, 'best-model.pt') 
            # print(acc_val)
#         step += 1
        
    print("Learning rate:",lr,"batch_size:", batch_size, "Accuracy", best_acc_val)
    # return best_acc_val


In [31]:
lr_list = [0.00001, 0.0001, 0.001]
batch_size = [16, 64, 256]

for lr in lr_list:
    for bs in batch_size:
        
        train(lr = lr, batch_size = bs)

Learning rate: 1e-05 batch_size: 16 Accuracy 0.448
Learning rate: 1e-05 batch_size: 64 Accuracy 0.448
Learning rate: 1e-05 batch_size: 256 Accuracy 0.402
Learning rate: 0.0001 batch_size: 16 Accuracy 0.483
Learning rate: 0.0001 batch_size: 64 Accuracy 0.454
Learning rate: 0.0001 batch_size: 256 Accuracy 0.48
Learning rate: 0.001 batch_size: 16 Accuracy 0.471
Learning rate: 0.001 batch_size: 64 Accuracy 0.46
Learning rate: 0.001 batch_size: 256 Accuracy 0.451


Epoch 0
Learning rate: 1e-05 batch_size: 32 Accuracy 0.28


In [ ]:
# classif = Classifier().cuda(0)
# classif.load_state_dict(torch.load("./best-model.pt"))

# z_val = classif(x_val)
# acc_val = ((torch.argmax(z_val, 1) == torch.argmax(y_val, 1)).float().mean()).item()
# acc_val

In [ ]:
# z_val = classif(x_val)



In [ ]:
# def evaluate(model, X, Y, params = ["acc"]):
#     results = []
#     batch_size = 100

#     predicted = []

#     X = X.cpu().numpy()
#     Y = Y.cpu()
#     # X = X

#     for i in range(int(len(X)/batch_size)):
#         s = i*batch_size
#         e = i*batch_size+batch_size

#         inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
#         pred = model(inputs)

#         predicted.append(pred.data.cpu().numpy())


#     inputs = Variable(torch.from_numpy(X).cuda(0))
#     predicted = model(inputs)

#     predicted = predicted.data.cpu().numpy()

#     for param in params:
#         if param == 'acc':
#             results.append(accuracy_score(Y, np.round(predicted)))
#         if param == "auc":
#             results.append(roc_auc_score(Y, predicted))
#         if param == "recall":
#             results.append(recall_score(Y, np.round(predicted)))
#         if param == "precision":
#             results.append(precision_score(Y, np.round(predicted)))
#         if param == "fmeasure":
#             precision = precision_score(Y, np.round(predicted))
#             recall = recall_score(Y, np.round(predicted))
#             results.append(2*precision*recall/ (precision+recall))
#     return results

In [ ]:
# batch_size = 64
# epochs = 5
# for epoch in range(epochs):  # loop over the dataset multiple times
#     # print ("\nEpoch ", epoch)

#     running_loss = 0.0
#     for i in range(int(len(X_train_eeg)/batch_size-1)):
#         s = i*batch_size
#         e = i*batch_size+batch_size
        
#         inputs = torch.from_numpy(X_train_eeg[s:e].numpy())
#         labels = torch.FloatTensor(np.array([y_train_eeg[s:e].numpy()]).T*1.0)
        # labels = torch.Tensor(labels.flatten())
        
    #     # # wrap them in Variable
    #     inputs, labels = Variable(inputs.cuda(0), requires_grad=True), Variable(labels.cuda(0), requires_grad=True)

    #     # # zero the parameter gradients
    #     optimizer.zero_grad()
        
    #     outputs = classif(inputs)
    #     outputs = torch.argmax(outputs, dim = 1).to(torch.float32)
    #     outputs = outputs.reshape((-1,1))
        
    #     # loss = criterion(outputs, labels)
    #     loss = F.cross_entropy(outputs, labels)
    #     loss.backward()
    #     optimizer.step()
    #     running_loss += loss.item()

    # # Validation accuracy
    # params = ["acc", "auc", "fmeasure"]
    # print (params)
    # print ("Training Loss ", running_loss)
    # print ("Train - ", evaluate(classif, X_train_eeg, y_train_eeg, params))
    # print ("Validation - ", evaluate(classif, X_val, y_val, params))
    # print ("Test - ", evaluate(net, X_test, y_test, params))

In [ ]:
# batch_size = 32

# for epoch in range(10):  # loop over the dataset multiple times
#     print ("\nEpoch ", epoch)

#     running_loss = 0.0
#     for i in range(int(len(X_train_eeg)/batch_size-1)):
#         s = i*batch_size
#         e = i*batch_size+batch_size

#         inputs = (X_train_eeg[s:e])
#         labels = torch.FloatTensor(np.array([y_train_eeg[s:e]]).T*1.0)

#         # wrap them in Variable
#         inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = classif(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()


#         optimizer.step()

#         running_loss += loss.item()

#     # # Validation accuracy
#     # params = ["acc", "auc", "fmeasure"]
#     # print (params)
#     # print ("Training Loss ", running_loss)
#     # print ("Train - ", evaluate(net, X_train, y_train, params))
#     # print ("Validation - ", evaluate(net, X_val, y_val, params))
#     # print ("Test - ", evaluate(net, X_test, y_test, params))

## Convolution Functions

In [ ]:
# from math import floor
# def compute_dim_Conv2D(Hin, 
#                 kernel_size=3, 
#                 stride=1,
#                 padding = 0,
#                 dilation = 1
#                 ):
#     return floor( (Hin + (2*padding) - (dilation*(kernel_size-1)) -1)/(stride) + 1 ) 

# def compute_dim_TransConv2D(Hin, 
#                         kernel_size=118, 
#                         stride=1,
#                         padding = 0,
#                         dilation = 1,
#                         output_padding = 0
        
#         ):
#     return ((Hin - 1)*stride) - (2*padding)+ (dilation*(kernel_size - 1)) + output_padding + 1


# def compute_maxpool_2d(Hin, 
#                         kernel_size=4, 
#                         padding = 0,
#                         dilation = 1        
#         ):

#     stride = kernel_size
#     return ( ( (Hin + (2*padding)-(dilation*(kernel_size - 1)) -1) / stride ) + 1)
 
